# Kubernetes

This is my compilation of tutorial [develop ML model with MLflow and deploy to Kubernetes](https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html) on the official mlflow site.

## Setup

In [21]:
%%bash
docker run -p 5000:5000 -dt --name mlflow_deploy --rm \
    ghcr.io/mlflow/mlflow \
    bash -c "mlflow server --host 0.0.0.0 --port 5000"

15f3ec9eec5513bf91736096c43a11f073649a0fca74975192bf34610369047d


In [18]:
import mlflow
import requests
import numpy as np
import subprocess
from sklearn import datasets, metrics
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
%%bash
docker stop mlflow_deploy

## Model

### Creating model

In [22]:
def eval_metrics(pred, actual):
    rmse = np.sqrt(metrics.mean_squared_error(actual, pred))
    mae = metrics.mean_absolute_error(actual, pred)
    r2 = metrics.r2_score(actual, pred)
    return rmse, mae, r2

# Set th experiment name
mlflow.set_experiment("wine-quality")

# Enable auto-logging to MLflow
mlflow.sklearn.autolog()

# Load wine quality dataset
X, y = datasets.load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Start a run and train a model
with mlflow.start_run(run_name="default-params") as run:
    run_id = run.info.run_id
    lr = ElasticNet()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    metr = eval_metrics(y_pred, y_test)

model_name = "wine-quality"
result = mlflow.register_model(
    f"runs:/{run_id}/model", model_name
)

2024/06/07 17:48:16 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.
Successfully registered model 'wine-quality'.
2024/06/07 17:48:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine-quality, version 1
Created version '1' of model 'wine-quality'.


### Check

In [23]:
start_api_command = f'MLFLOW_TRACKING_URI=http://localhost:5000 mlflow models serve --no-conda -m "models:/{model_name}/1" -p 4242'
ans = subprocess.run([
    'gnome-terminal', '--', 'bash', '-c', start_api_command
])

In [24]:
url = "http://127.0.0.1:4242/invocations"
headers = {
    'Content-Type': 'application/json',
}
data = {
    "inputs": [[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]]
}
response = requests.post(url, headers=headers, json=data)

print(response.text)

{"predictions": [0.4122446447044683]}


## Kubernetes

In [25]:
!kubectl create namespace mlflow-kserve-test

2024/06/07 17:54:43.221117 system_key.go:131: cannot determine nfs usage in generateSystemKey: cannot parse /etc/fstab: expected between 3 and 6 fields, found 7
namespace/mlflow-kserve-test created
